# Nested function Calling with Mistral AI API


Function calling allows Mistral models to connect to external tools. By integrating Mistral models with external tools such as user defined functions or APIs, users can easily build applications catering to specific use cases and practical problems. In this guide, for instance, we wrote two functions for tracking payment status and payment date. We can use these two tools to provide answers for payment-related queries.

At a glance, there are four steps with function calling:

- User: specify tools and query
- Model: Generate function arguments if applicable
- User: Execute function to obtain tool results
- Model: Generate final answer

This notebook is inspired by the [Mistral AI](https://www.mistral.ai/) youtube video on [Function Calling with Mistral AI](https://www.youtube.com/watch?v=eOo4GfHj3ZE&t=214s). Here, we will see how to call a function using the Mistral large model and the Mistral AI API.

In [80]:
# Installing the necessary packages 

!pip install pandas mistralai

## Step 1. User: specify tools and query

### Tools

Users can define all the necessary tools for their use cases. Here we define functions that will be used to answer the user's query.

In [81]:
# defintion of the function

def special_calc(a: int, b: int) -> int:
    return a * 23 + 3 * b + 2003

def random_calc(a: int, b: int) -> int:
    return a * 3 + 2 * b + 100

def dual_add(a: int, b: int, c: int, d: int) -> int:
    a = a + b
    c = c + d
    return a,c

- In order for Mistral models to understand the functions, we need to outline the function specifications with a JSON schema. Specifically, we need to describe the type, function name, function description, function parameters, and the required parameter for the function. Since we have three functions, we will define three JSON schemas.

In [82]:
# definition of the tools

tools = [
    {
        "type": "function",
        "function": {
            "name": "special_calc",
            "description": "given a and b, return a * 23 + 3 * b + 2003",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "The first number.",
                    },
                    "b": {
                        "type": "integer",
                        "description": "The second number.",
                    }
                },
                "required": ["a", "b"],
        },
    },
    },
    {
        "type": "function",
        "function": {
            "name": "random_calc",
            "description": "given a and b, return a * 3 + 2 * b + 100",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "The first number.",
                    },
                    "b": {
                        "type": "integer",
                        "description": "The second number.",
                    }
                },
                "required": ["a", "b"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "dual_add",
            "description": "given a, b, c, d, return a+b, c+d",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "The first number.",
                    },
                    "b": {
                        "type": "integer",
                        "description": "The second number.",
                    },
                    "c": {
                        "type": "integer",
                        "description": "The third number.",
                    },
                    "d": {
                        "type": "integer",
                        "description": "The fourth number.",
                    }
                },
                "required": ["a", "b", "c", "d"],
            },
        },
    }

]

- Then we organize our three functions into a dictionary where keys represent the function name, and values are the function names. This allows us to call each function based on its function name.

In [83]:
import functools


names_to_functions = ({
    'special_calc': special_calc,
    'random_calc': random_calc,
    'dual_add': dual_add
})


### User query

Suppose a user asks to perform our dual addition, a standalone LLM would not be able to answer this question correctly because it is something we came up with. But what if we have an exact tool we can use to answer this question? We could potentially provide an answer!

In [84]:
messages = [{"role": "user", "content": "What is the result of the special calculation of the dual addition of 3 and 4 and 5 and 6?"}] 

## Step 2. Model: Generate function arguments

How do Mistral models know about these functions and know which function to use? We provide both the user query and the tools specifications to Mistral models. The goal in this step is not for the Mistral model to run the function directly. 
It’s to:
 1) determine the appropriate function to use based on the user query
 2) identify if there is any essential information missing for a function to run
 3) generate necessary arguments for the chosen function 

In [96]:
from mistralai import Mistral

api_key ="xxxxxxxxxxxxxxxx"
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

response = client.chat.complete(
    model = model,
    messages = messages,
    tools = tools,
    tool_choice = "any",
)

response



ChatCompletionResponse(id='966ce8dca12b44a7b659c9f1eddc7df0', object='chat.completion', model='mistral-large-latest', usage=UsageInfo(prompt_tokens=456, completion_tokens=27, total_tokens=483), created=1728382226, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='special_calc', arguments='{"a": 7, "b": 11}'), id='ZGeqHQ7t0', type='function')], prefix=False, role='assistant'), finish_reason='tool_calls')])

In [97]:
messages.append(response.choices[0].message) 

messages

[{'role': 'user',
  'content': 'What is the result of the special calculation of the dual addition of 3 and 4 and 5 and 6?'},
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='dual_add', arguments='{"a": 3, "b": 4, "c": 5, "d": 6}'), id='4nZ0s8Yxj', type='function')], prefix=False, role='assistant'),
 {'role': 'tool',
  'name': 'dual_add',
  'content': '(7, 11)',
  'tool_call_id': '4nZ0s8Yxj'},
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='special_calc', arguments='{"a": 7, "b": 11}'), id='ZGeqHQ7t0', type='function')], prefix=False, role='assistant')]

## Step 3. User: Execute function to obtain tool results

How do we execute the function? Currently, it is the user’s responsibility to execute these functions and the function execution lies on the user side. In the future, we may introduce some helpful functions that can be executed server-side.

Let’s extract some useful function information from model response including function_name and function_params.

In [98]:
import json


def process_function_calls(response, names_to_functions, messages):
    """
    Process multiple or nested function calls from the model response.
    """
    while True:
        # Check if there are function calls in the response
        tool_calls = response.choices[0].message.tool_calls
        if not tool_calls:
            break

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_params = json.loads(tool_call.function.arguments)
            

            # Execute the function
            function_result = names_to_functions[function_name](**function_params)
            print(f"Executed {function_name} with result: {function_result}")

            # Append the result to the messages
            messages.append({
                "role": "tool",
                "name": function_name,
                "content": str(function_result),
                "tool_call_id": tool_call.id
            })

        # Continue calling the model with updated messages
        response = client.chat.complete(model=model, messages=messages)

    return response

messages

response = process_function_calls(response, names_to_functions, messages)

response

Executed special_calc with result: 2197


ChatCompletionResponse(id='efcac68c0c414c989afda07cee81f7c6', object='chat.completion', model='mistral-large-latest', usage=UsageInfo(prompt_tokens=177, completion_tokens=29, total_tokens=206), created=1728382272, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='The result of the special calculation of the dual addition of 3 and 4 and 5 and 6 is 2197.', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')])

## Step 4. Model: Generate final answer

We can now provide the output from the tools to Mistral models, and in return, the Mistral model can produce a customised final response for the specific user.

In [99]:
response.choices[0].message.content

'The result of the special calculation of the dual addition of 3 and 4 and 5 and 6 is 2197.'